In [1]:
### Load the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, export_graphviz,DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_absolute_error
from sklearn import tree,ensemble
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
data=pd.read_csv("C:/Users/yugandhar/Downloads/modules/employee_data.csv")

In [3]:
data.head(10)

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure
0,221,engineering,NaN,0.932868,4,NaN,low,0.829896,Left,5.0
1,232,support,NaN,NaN,3,NaN,low,0.834544,Employed,2.0
2,184,sales,NaN,0.788830,3,NaN,medium,0.834988,Employed,3.0
3,206,sales,NaN,0.575688,4,NaN,low,0.424764,Employed,2.0
4,249,sales,NaN,0.845217,3,NaN,low,0.779043,Employed,3.0
5,140,sales,NaN,0.589097,4,NaN,medium,0.660020,Employed,4.0
6,121,sales,1.0,0.625399,3,NaN,low,0.835571,Employed,3.0
7,150,engineering,NaN,0.644586,4,NaN,low,0.796683,Employed,3.0
8,215,engineering,1.0,0.524114,3,NaN,medium,0.715005,Employed,7.0
9,269,support,NaN,0.909364,5,NaN,medium,0.994037,Employed,2.0


In [4]:
data.describe()

,avg_monthly_hrs,filed_complaint,last_evaluation,n_projects,recently_promoted,satisfaction,tenure
count,14249.000000,2058.0,12717.000000,14249.000000,300.0,14068.000000,14068.000000
mean,199.795775,1.0,0.718477,3.773809,1.0,0.621295,3.497228
std,50.998714,0.0,0.173062,1.253126,0.0,0.250469,1.460917
min,49.000000,1.0,0.316175,1.000000,1.0,0.040058,2.000000
25%,155.000000,1.0,0.563866,3.000000,1.0,0.450390,3.000000
50%,199.000000,1.0,0.724939,4.000000,1.0,0.652527,3.000000
75%,245.000000,1.0,0.871358,5.000000,1.0,0.824951,4.000000
max,310.000000,1.0,1.000000,7.000000,1.0,1.000000,10.000000


In [5]:
data.shape

(14249, 10)

In [6]:
data.dtypes

avg_monthly_hrs        int64
department            object
filed_complaint      float64
last_evaluation      float64
n_projects             int64
recently_promoted    float64
salary                object
satisfaction         float64
status                object
tenure               float64
dtype: object

In [7]:
data.department.unique() 

array(['engineering', 'support', 'sales', 'IT', 'product', 'marketing',
       'temp', 'procurement', 'finance', nan, 'management',
       'information_technology', 'admin'], dtype=object)

In [8]:
data.salary.unique()

array(['low', 'medium', 'high'], dtype=object)

In [9]:
data.isnull().sum()

avg_monthly_hrs          0
department             709
filed_complaint      12191
last_evaluation       1532
n_projects               0
recently_promoted    13949
salary                   0
satisfaction           181
status                   0
tenure                 181
dtype: int64

In [10]:
def inspect_data(data):
    return pd.DataFrame({"Data Type":data.dtypes,"No of Levels":data.apply(lambda x: x.nunique(),axis=0), "Levels":data.apply(lambda x: str(x.unique()),axis=0)})
inspect_data(data)

,Data Type,No of Levels,Levels
avg_monthly_hrs,int64,249,[221 232 184 206 249 140 121 150 215 269 147 1...
department,object,12,['engineering' 'support' 'sales' 'IT' 'product...
filed_complaint,float64,1,[nan 1.0]
last_evaluation,float64,12359,[0.932867640125 nan 0.7888298419059999 ... 0.8...
n_projects,int64,7,[4 3 5 2 6 1 7]
recently_promoted,float64,1,[nan 1.0]
salary,object,3,['low' 'medium' 'high']
satisfaction,float64,13710,[0.829896195856 0.83454379486 0.83498761146 .....
status,object,2,['Left' 'Employed']
tenure,float64,8,[5.0 2.0 3.0 4.0 7.0 6.0 nan 10.0 8.0]


In [11]:
#function for missing values in columns in data
def missing_coldata(df):
    missin_col = pd.DataFrame(round(df.isnull().sum().sort_values(ascending=False)/len(df.index)*100,1),
                              columns=['% of missing value'])
    missin_col['Count of Missing Values'] = df.isnull().sum()
    return missin_col

In [12]:
missing_coldata(data)

,% of missing value,Count of Missing Values
recently_promoted,97.9,13949
filed_complaint,85.6,12191
last_evaluation,10.8,1532
department,5.0,709
tenure,1.3,181
satisfaction,1.3,181
status,0.0,0
salary,0.0,0
n_projects,0.0,0
avg_monthly_hrs,0.0,0


In [13]:
data['department'].value_counts()

sales                     3923
engineering               2592
support                   2122
IT                        1161
product                    860
marketing                  819
finance                    728
management                 595
information_technology     209
temp                       181
admin                      176
procurement                174
Name: department, dtype: int64

In [14]:
#data['filed_complaint']=data['filed_complaint'].replace(np.NaN,0)

In [15]:
#data['recently_promoted']=data['recently_promoted'].replace(np.NaN,0)

In [16]:
data=data.drop(["filed_complaint","recently_promoted"],axis=1)

In [17]:
data['tenure']=data['tenure'].replace(np.NaN,0)

In [18]:
data['department']=data['department'].replace(np.NaN,'sales')

In [19]:
data['department'].value_counts()

sales                     4632
engineering               2592
support                   2122
IT                        1161
product                    860
marketing                  819
finance                    728
management                 595
information_technology     209
temp                       181
admin                      176
procurement                174
Name: department, dtype: int64

In [20]:
cat_cols=["department","salary","n_projects","tenure","status"]
num_cols=["last_evaluation","satisfaction","avg_monthly_hrs"]

In [21]:
data[cat_cols] = data[cat_cols].apply(lambda x: x.astype('category'))
data[num_cols] = data[num_cols].apply(lambda x: x.astype('float'))
data.dtypes

avg_monthly_hrs     float64
department         category
last_evaluation     float64
n_projects         category
salary             category
satisfaction        float64
status             category
tenure             category
dtype: object

In [22]:
data.head()

,avg_monthly_hrs,department,last_evaluation,n_projects,salary,satisfaction,status,tenure
0,221.0,engineering,0.932868,4,low,0.829896,Left,5.0
1,232.0,support,NaN,3,low,0.834544,Employed,2.0
2,184.0,sales,0.788830,3,medium,0.834988,Employed,3.0
3,206.0,sales,0.575688,4,low,0.424764,Employed,2.0
4,249.0,sales,0.845217,3,low,0.779043,Employed,3.0


In [23]:
## Convert Categorical Columns to Dummies
cat_cols=["department","salary","n_projects","tenure","status"]
data = pd.get_dummies(data,columns=cat_cols,drop_first=True,)

In [24]:
data.isnull().sum()

avg_monthly_hrs                         0
last_evaluation                      1532
satisfaction                          181
department_admin                        0
department_engineering                  0
department_finance                      0
department_information_technology       0
department_management                   0
department_marketing                    0
department_procurement                  0
department_product                      0
department_sales                        0
department_support                      0
department_temp                         0
salary_low                              0
salary_medium                           0
n_projects_2                            0
n_projects_3                            0
n_projects_4                            0
n_projects_5                            0
n_projects_6                            0
n_projects_7                            0
tenure_2.0                              0
tenure_3.0                        

In [25]:
data.shape

(14249, 31)

In [26]:
data.head()



,avg_monthly_hrs,last_evaluation,satisfaction,department_admin,department_engineering,department_finance,department_information_technology,department_management,department_marketing,department_procurement,...,n_projects_7,tenure_2.0,tenure_3.0,tenure_4.0,tenure_5.0,tenure_6.0,tenure_7.0,tenure_8.0,tenure_10.0,status_Left
0,221.0,0.932868,0.829896,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,232.0,NaN,0.834544,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,184.0,0.788830,0.834988,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,206.0,0.575688,0.424764,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,249.0,0.845217,0.779043,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [27]:
data.columns

Index(['avg_monthly_hrs', 'last_evaluation', 'satisfaction',
       'department_admin', 'department_engineering', 'department_finance',
       'department_information_technology', 'department_management',
       'department_marketing', 'department_procurement', 'department_product',
       'department_sales', 'department_support', 'department_temp',
       'salary_low', 'salary_medium', 'n_projects_2', 'n_projects_3',
       'n_projects_4', 'n_projects_5', 'n_projects_6', 'n_projects_7',
       'tenure_2.0', 'tenure_3.0', 'tenure_4.0', 'tenure_5.0', 'tenure_6.0',
       'tenure_7.0', 'tenure_8.0', 'tenure_10.0', 'status_Left'],
      dtype='object')

In [28]:
## Split the data into X and y
X = data.copy().drop("status_Left",axis=1)
y = data["status_Left"]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [30]:
## Print the shape of X_train, X_test, y_train, y_test
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9974, 30)
(4275, 30)
(9974,)
(4275,)


In [31]:
X_test.isnull().sum()

avg_monthly_hrs                        0
last_evaluation                      473
satisfaction                          54
department_admin                       0
department_engineering                 0
department_finance                     0
department_information_technology      0
department_management                  0
department_marketing                   0
department_procurement                 0
department_product                     0
department_sales                       0
department_support                     0
department_temp                        0
salary_low                             0
salary_medium                          0
n_projects_2                           0
n_projects_3                           0
n_projects_4                           0
n_projects_5                           0
n_projects_6                           0
n_projects_7                           0
tenure_2.0                             0
tenure_3.0                             0
tenure_4.0      

In [32]:
X_train['satisfaction'].fillna(X_train['satisfaction'].mean(),inplace=True)

C:\Users\yugandhar\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [33]:
X_train['last_evaluation'].fillna(X_train['last_evaluation'].mean(),inplace=True)

In [34]:
X_train.isnull().sum()

avg_monthly_hrs                      0
last_evaluation                      0
satisfaction                         0
department_admin                     0
department_engineering               0
department_finance                   0
department_information_technology    0
department_management                0
department_marketing                 0
department_procurement               0
department_product                   0
department_sales                     0
department_support                   0
department_temp                      0
salary_low                           0
salary_medium                        0
n_projects_2                         0
n_projects_3                         0
n_projects_4                         0
n_projects_5                         0
n_projects_6                         0
n_projects_7                         0
tenure_2.0                           0
tenure_3.0                           0
tenure_4.0                           0
tenure_5.0               

In [35]:
X_test.isnull().sum()

avg_monthly_hrs                        0
last_evaluation                      473
satisfaction                          54
department_admin                       0
department_engineering                 0
department_finance                     0
department_information_technology      0
department_management                  0
department_marketing                   0
department_procurement                 0
department_product                     0
department_sales                       0
department_support                     0
department_temp                        0
salary_low                             0
salary_medium                          0
n_projects_2                           0
n_projects_3                           0
n_projects_4                           0
n_projects_5                           0
n_projects_6                           0
n_projects_7                           0
tenure_2.0                             0
tenure_3.0                             0
tenure_4.0      

In [36]:
X_test['satisfaction'].fillna(X_train['satisfaction'].mean(),inplace=True)

In [37]:
X_test['last_evaluation'].fillna(X_train['last_evaluation'].mean(),inplace=True)

In [38]:
X_test.isnull().sum()

avg_monthly_hrs                      0
last_evaluation                      0
satisfaction                         0
department_admin                     0
department_engineering               0
department_finance                   0
department_information_technology    0
department_management                0
department_marketing                 0
department_procurement               0
department_product                   0
department_sales                     0
department_support                   0
department_temp                      0
salary_low                           0
salary_medium                        0
n_projects_2                         0
n_projects_3                         0
n_projects_4                         0
n_projects_5                         0
n_projects_6                         0
n_projects_7                         0
tenure_2.0                           0
tenure_3.0                           0
tenure_4.0                           0
tenure_5.0               

In [39]:
X_train.head()

,avg_monthly_hrs,last_evaluation,satisfaction,department_admin,department_engineering,department_finance,department_information_technology,department_management,department_marketing,department_procurement,...,n_projects_6,n_projects_7,tenure_2.0,tenure_3.0,tenure_4.0,tenure_5.0,tenure_6.0,tenure_7.0,tenure_8.0,tenure_10.0
3896,174.0,0.464176,0.545810,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8584,242.0,0.635318,0.659734,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7945,225.0,0.709918,0.286291,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
11397,109.0,0.688718,0.687016,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11012,103.0,0.718056,0.621544,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
X_test.head()

,avg_monthly_hrs,last_evaluation,satisfaction,department_admin,department_engineering,department_finance,department_information_technology,department_management,department_marketing,department_procurement,...,n_projects_6,n_projects_7,tenure_2.0,tenure_3.0,tenure_4.0,tenure_5.0,tenure_6.0,tenure_7.0,tenure_8.0,tenure_10.0
3179,224.0,0.421574,0.293154,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
6360,184.0,0.834351,0.972236,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5208,180.0,0.708075,0.511586,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9277,243.0,0.718056,0.904509,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
7044,169.0,0.923479,0.780903,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [41]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train.iloc[:,:3])

X_train.iloc[:,:3]=scaler.transform(X_train.iloc[:,:3])
X_test.iloc[:,:3]=scaler.transform(X_test.iloc[:,:3])

C:\Users\yugandhar\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\yugandhar\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [42]:
X_train.head()

,avg_monthly_hrs,last_evaluation,satisfaction,department_admin,department_engineering,department_finance,department_information_technology,department_management,department_marketing,department_procurement,...,n_projects_6,n_projects_7,tenure_2.0,tenure_3.0,tenure_4.0,tenure_5.0,tenure_6.0,tenure_7.0,tenure_8.0,tenure_10.0
3896,-0.496136,-1.555976e+00,-3.046429e-01,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8584,0.837232,-5.070809e-01,1.536204e-01,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7945,0.503890,-4.987618e-02,-1.348566e+00,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
11397,-1.770679,-1.798083e-01,2.633623e-01,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11012,-1.888329,6.123873e-15,2.679542e-15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
X_test.head()

,avg_monthly_hrs,last_evaluation,satisfaction,department_admin,department_engineering,department_finance,department_information_technology,department_management,department_marketing,department_procurement,...,n_projects_6,n_projects_7,tenure_2.0,tenure_3.0,tenure_4.0,tenure_5.0,tenure_6.0,tenure_7.0,tenure_8.0,tenure_10.0
3179,0.484281,-1.817074e+00,-1.320959,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
6360,-0.300053,7.127466e-01,1.410668,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5208,-0.378486,-6.117501e-02,-0.442312,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9277,0.856840,-5.035184e-14,1.138232,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
7044,-0.594178,1.258990e+00,0.641026,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [44]:
######logistic regression

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [46]:
lrc = LogisticRegression()

lrc.fit(X_train,y_train)

y_pred_train_lrc = lrc.predict(X_train)
y_pred_test_lrc = lrc.predict(X_test)

C:\Users\yugandhar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [47]:
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score

#logistic_model = LogisticRegression(class_weight='balanced')
#logistic_model.fit(x_train,y_train)
#trainy_pred = logistic_model.predict(x_train)
#testy_pred = logistic_model.predict(x_validation)
print(classification_report(y_train,y_pred_train_lrc))
print(classification_report(y_test,y_pred_test_lrc))


              precision    recall  f1-score   support

           0       0.94      0.93      0.93      7652
           1       0.77      0.81      0.79      2322

    accuracy                           0.90      9974
   macro avg       0.85      0.87      0.86      9974
weighted avg       0.90      0.90      0.90      9974

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      3205
           1       0.77      0.79      0.78      1070

    accuracy                           0.89      4275
   macro avg       0.85      0.85      0.85      4275
weighted avg       0.89      0.89      0.89      4275



In [48]:
print('Accuracy of lrc classofier on a test set: {:.2f}'.format(lrc.score(X_test,y_test)))

Accuracy of lrc classofier on a test set: 0.89


In [49]:
print('Accuracy of lrc classofier on a train set: {:.2f}'.format(lrc.score(X_train,y_train)))

Accuracy of lrc classofier on a train set: 0.90


In [50]:
ytprobs = lrc.predict_proba(X_train)
preds = ytprobs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_train, y_pred_train_lrc)
roc_auc = metrics.auc(fpr, tpr)

ytprobs1 = lrc.predict_proba(X_test)
preds1 = ytprobs1[:,1]
fpr1, tpr1, threshold1 = metrics.roc_curve(y_test, y_pred_test_lrc)
roc_auc1 = metrics.auc(fpr1, tpr1)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.plot(fpr1, tpr1, 'r', label = 'AUC = %0.2f' % roc_auc1)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

NameError: name 'metrics' is not defined

In [ ]:
from sklearn.model_selection import learning_curve
train_size, train_score, test_score = learning_curve(estimator=lrc, X=X_train, y=y_train, cv=10 )

train_score_m = np.mean(train_score, axis=1)
test_score_m = np.mean(test_score, axis=1)

plt.plot(train_size, train_score_m, 'o-', color="b")
plt.plot(train_size, test_score_m, 'o-', color="r")
plt.legend(('Training score', 'Test score'), loc='best')
plt.xlabel("Training Samples")
plt.ylabel("Score")
plt.title("Learning Curve for RF")
plt.grid()

plt.show()

In [ ]:
X_train.columns

In [ ]:
sns.lmplot(x='avg_monthly_hrs',y='',data=X_train)

In [ ]:
#####decision tree

In [ ]:
dtc = DecisionTreeClassifier()

dtc.fit(X_train,y_train)

y_pred_train_dtc = dtc.predict(X_train)
y_pred_test_dtc = dtc.predict(X_test)

In [ ]:

print(classification_report(y_train,y_pred_train_dtc))
print(classification_report(y_test,y_pred_test_dtc))

pred_rfc_train=accuracy_score(y_train, y_pred_train_dtc)
pred_rfc_test=accuracy_score(y_test, y_pred_test_dtc)

In [ ]:
######svc

In [ ]:
svc = SVC()

svc.fit(X_train,y_train)

y_pred_train_svc = svc.predict(X_train)
y_pred_test_svc = svc.predict(X_test)

In [ ]:

print(classification_report(y_train,y_pred_train_svc))
print(classification_report(y_test,y_pred_test_svc))

pred_rfc_train=accuracy_score(y_train, y_pred_train_svc)
pred_rfc_test=accuracy_score(y_test, y_pred_test_svc)

In [ ]:
######random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X = X_train,y = y_train)

In [ ]:
train_predictions = rfc.predict(X_train)
test_predictions = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score
print("\nTrain accuracy",accuracy_score(y_train,train_predictions))
#print("\nTrain f1-score for class '0'",f1_score(y_train,train_predictions,pos_label="0"))
#print("\nTrain f1-score for class '1'",f1_score(y_train,train_predictions,pos_label="1"))

In [ ]:
print("\nTest accuracy",accuracy_score(y_test,test_predictions))
#print("\nTest f1-score for class '1'",f1_score(y_test,test_predictions,pos_label="1"))
#print("\nTest f1-score for class '0'",f1_score(y_test,test_predictions,pos_label="0"))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,test_predictions)

In [ ]:

print(classification_report(y_train,train_predictions))
print(classification_report(y_test,test_predictions))

pred_rfc_train=accuracy_score(y_train, train_predictions)
pred_rfc_test=accuracy_score(y_test, test_predictions)

In [ ]:
#########knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=3,weights="distance",algorithm="brute")
knn_classifier.fit(X_train, y_train)

In [ ]:
pred_train = knn_classifier.predict(X_train) 
pred_test = knn_classifier.predict(X_test)

In [ ]:
cm_test = confusion_matrix(y_pred=pred_test, y_true=y_test)

print(cm_test)

In [ ]:
# Accuracy: 

sum(np.diag(cm_test))/np.sum(cm_test)

# np.trace(cm_test)/np.sum(cm_test)

In [ ]:
print("Accuracy on train is:",accuracy_score(y_train,pred_train))
print("Accuracy on test is:",accuracy_score(y_test,pred_test))

In [ ]:
######adaboost

In [ ]:
ABclassifier = ensemble.AdaBoostClassifier()
ABclassifier.fit(X_train,y_train)
AB_train_preds=ABclassifier.predict(X_train)
AB_test_preds = ABclassifier.predict(X_test)
print("train_confusion Matrix")
print(confusion_matrix(y_train,AB_train_preds))
print("test_confusion Matrix")
print(confusion_matrix(y_test,AB_test_preds))
print("Classification Report")
print(classification_report(y_test,AB_test_preds))
print("Classification Report")
print(classification_report(y_train,AB_train_preds))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train,AB_train_preds))
print(accuracy_score(y_test,AB_test_preds))

In [ ]:
###########xgboost

In [ ]:
pip install xgboost

In [ ]:
from xgboost import  XGBClassifier
X_classifier = XGBClassifier()
X_classifier.fit(X_train,y_train)

In [ ]:
xgboost_train_preds=X_classifier.predict(X_train)
xgboost_test_preds=X_classifier.predict(X_test)                     

In [ ]:
print('Train Accuracy =',X_classifier.score(X_train, xgboost_train_preds))
print('Test Accuracy =',X_classifier.score(X_test, xgboost_test_preds))

In [ ]:
print("Train - Confusion Matrix")
print(confusion_matrix(y_train,X_classifier.predict(X_train)))
print("Test - Confusion Matrix")
print(confusion_matrix(y_test,X_classifier.predict(X_test)))

In [ ]:
print("Classification Report")
print(classification_report(y_test,xgboost_test_preds))
print("Classification Report")
print(classification_report(y_train,xgboost_train_preds))

In [ ]:
######navie bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
nb_pred_test = gaussian.predict(X_test)
nb_pred_train = gaussian.predict(X_train)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train,nb_pred_train))
print(accuracy_score(y_test,nb_pred_test))
#acc_gaussian = round(gaussian.score(X_train, y_train) * 100, 2)
#acc_gaussian
#print(accuracy_score(y_test,nb))